In [1]:
from db.data import Data
import gurobipy as gp
from gurobipy import GRB
from scipy.special import comb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplleaflet

In [2]:
data = Data()

demand = data.get_demand()
waiting = data.get_waiting()
distances = data.get_distances()

RADIUS = 3000
MAX_OFFICERS = 10
ALPHA = 1
LAMBDA = 1.35
SERVICE_TIME = 1

In [3]:
# list(set(waiting['name']) - set(distances['WPname']))

In [4]:
distances['in_range'] = (distances['distance'] <= RADIUS).astype(int)
distances = distances.pivot(index='WPname',columns='DAuid',values='in_range')
# distances = distances.loc[:, (distances.sum() > 0)]
distances.drop('Holland Park', inplace=True)

In [5]:
DAuid = list(distances.columns)
demand = demand[demand['DAuid'].isin(DAuid)]

In [6]:
distances.head()

DAuid,59151838,59151839,59151840,59151841,59151842,59151843,59151844,59151845,59151846,59151847,...,59154008,59154009,59154010,59154011,59154020,59154021,59154022,59154023,59154024,59154025
WPname,,,,,,,,,,,,,,,,,,,,,
APH Matthew Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Aria Banquet Convention Center,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Bear Creek Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Blumsen Park,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,1,0,0,0
Bolivar Park,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
coverage = {}
for index, row in distances.iterrows():
    coverage[index] = [set(row[row==1].keys())]
    
pop_dict = pd.Series(demand.population_val.values,index=(demand.DAuid)).to_dict()

In [8]:
crime = data.get_crime()
crime

,type,location_string,month,year,file_number,lon,lat,closest_demand,distance
0,Break and Enter - Business,11000 124TH ST,1,2020,20-680,-122.878468,49.202197,59152187,1.519689
1,Break and Enter - Business,13900 104TH AVE,1,2020,20-1128,-122.836933,49.191220,59152204,1.724349
2,Break and Enter - Business,11100 SCOTT RD,1,2020,20-1173,-122.877527,49.203288,59152187,2.838619
3,Break and Enter - Business,10500 UNIVERSITY DR,1,2020,20-1191,-122.850623,49.193487,59153386,1.849879
4,Break and Enter - Business,10400 138TH ST,1,2020,20-1298,-122.837520,49.150378,59151980,3.695559
...,...,...,...,...,...,...,...,...,...
15377,Fatal/Injury Collision,152ND ST / 20TH AVE,12,2020,20-194236,-122.801149,49.038372,59152656,1.849300
15378,Fatal/Injury Collision,2200 160TH ST,12,2020,20-195879,-122.778445,49.042048,59152667,5.264139
15379,Fatal/Injury Collision,13900 16TH AVE,12,2020,20-197230,-122.837383,49.031155,59152607,1.741249
15380,Fatal/Injury Collision,1700 KING GEORGE BLVD,12,2020,20-197622,-122.777242,49.032925,59152669,2.159922


In [9]:
#weigh each crime according to maximum sentence
incident_weight = {'Break and Enter - Business':3,
 'Break and Enter - Residence':5,
 'Fatal/Injury Collision':5,
 'Graffiti':2,
 'Shoplifting':1,
 'Theft from Motor Vehicle':4,
 'Theft of Motor Vehicle':4}
crime['weight'] = crime['type'].replace(incident_weight)

In [10]:
crime = crime[crime['distance'] < 200] #removing errors
crime = demand.set_index('DAuid').join(crime.groupby('closest_demand').sum()['weight']).reset_index().rename(columns={'weight':'incidents'})
crime['incidents'] = crime['incidents'].replace(np.nan, 0)
crime.head()

,DAuid,population_val,DAarea,lat,lon,FEDcode,FEDName,incidents
0,59151838,941.0,2.5917,49.187082,-122.740379,12,Fleetwood/Port Kells,70.0
1,59151839,303.0,2.7945,49.169710,-122.745703,12,Fleetwood/Port Kells,53.0
2,59151840,801.0,0.7341,49.173207,-122.765197,12,Fleetwood/Port Kells,90.0
3,59151841,731.0,0.2013,49.175556,-122.772461,12,Fleetwood/Port Kells,88.0
4,59151842,881.0,0.3040,49.172899,-122.775759,12,Fleetwood/Port Kells,34.0


In [11]:
regions, population = gp.multidict(pop_dict)
waits, covered = gp.multidict(coverage)

prob = [comb(MAX_OFFICERS,k)*((0.6)**k)*((0.4)**(MAX_OFFICERS-k)) for k in range(0,MAX_OFFICERS+1)]

In [12]:
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information

Academic license - for non-commercial use only - expires 2021-05-08
Using license file /Users/rbarket/gurobi.lic


In [13]:
wait_size = [(name,size) for name in list(waiting['name']) for size in range(0,MAX_OFFICERS+1)]
demand_size = [(DAuid,size) for DAuid in list(distances.columns) for size in range(0,MAX_OFFICERS+1)]

In [14]:
#variable about where to place officers
build = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(demand_size, vtype=GRB.BINARY, name="Is_covered")
build_change = m.addVars(wait_size, vtype=GRB.BINARY, name="wait_change")

In [15]:
#sum demand points only where officers can reach
for size in range(0,MAX_OFFICERS+1):
    m.addConstrs((gp.quicksum(build[wait,size] for wait in waits if r in covered[wait]) >= is_covered[r,size]
                            for r in regions), name="Build2cover")

In [16]:
#only allow max officers
for size in range(0,MAX_OFFICERS+1):
    m.addConstr(gp.quicksum(build[wait,size] for wait in waits) == size, name="officers")

In [17]:
#Control waiting site change
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build[wait,size] - build[wait,size+1] <= build_change[wait,size] for wait in waits), name="cease_wait")

In [18]:
#Max amount of location changes
for size in range(1,MAX_OFFICERS):
    m.addConstrs( (build_change[wait,size] <= ALPHA for wait in waits), name="cease_wait")

In [19]:
obj = gp.quicksum(prob[k]*population[r]*is_covered[r,k] for r in regions for k in range(1,MAX_OFFICERS+1))

In [20]:
m.setObjective(obj, GRB.MAXIMIZE)

In [21]:
m.optimize()

In [22]:
WPnames = []
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        if tower[1] == MAX_OFFICERS:
            WPnames.append(tower[0])
            print(f"\n Place officer at location {tower}.")
WPnames = list(set(WPnames))


 Place officer at location ('Cedar Hills Shopping Center', 10).

 Place officer at location ('Srawberry Hill Shopping Centre Shopping Mall', 10).

 Place officer at location ('TE Scott Park', 10).

 Place officer at location ('Panorama Park', 10).

 Place officer at location ('Hawthorne Park', 10).

 Place officer at location ('Fraser Heights Park', 10).

 Place officer at location ('Maple Green Park', 10).

 Place officer at location ('Bonnie Schrenk Park', 10).

 Place officer at location ('Hillcrest Park', 10).

 Place officer at location ('South Surrey RCMP', 10).


In [23]:
waiting_plot = waiting[waiting['name'].isin( WPnames )]
    # plt.plot(demand['lon'], demand['lat'], 'rs')
plt.plot(waiting_plot['lon'].astype(np.float64), waiting_plot['lat'].astype(np.float64), 'bs')
mplleaflet.show()

In [37]:
#STATIC CASE
MAX_OFFICERS=10
m = gp.Model("SPMARP")
m.Params.LogToConsole = 0 #dont print model information
#variable about where to place officers
build = m.addVars(list(waiting['name']), vtype=GRB.BINARY, name="wait_points")
is_covered = m.addVars(list(distances.columns), vtype=GRB.BINARY, name="Is_covered")

#sum demand points only where officers placed
m.addConstrs((gp.quicksum(build[t] for t in waits if r in covered[t]) >= is_covered[r]
                        for r in regions), name="Build2cover")

#only allow max officers
m.addConstr(gp.quicksum(build[t] for t in waits) == MAX_OFFICERS, name="officers")

m.setObjective(gp.quicksum(is_covered[r]*population[r] for r in regions), GRB.MAXIMIZE)

m.optimize()

print(f'model size={MAX_OFFICERS}')
WPnames2 = []
for tower in build.keys():
    if (abs(build[tower].x) > 1e-6):
        WPnames2.append(tower)
        print(f"\n Place officer at location {tower}.")
WPnames2 = list(set(WPnames2))

total_population = 0

for region in regions:
    total_population += population[region]

percent = round(100*m.objVal/total_population, 2)

print(f"\n The population coverage associated to the patrol officer distribution is: {percent} %\n\n")   

model size=10

 Place officer at location Cedar Hills Shopping Center.

 Place officer at location Srawberry Hill Shopping Centre Shopping Mall.

 Place officer at location TE Scott Park.

 Place officer at location Panorama Park.

 Place officer at location Hawthorne Park.

 Place officer at location Fraser Heights Park.

 Place officer at location Maple Green Park.

 Place officer at location Bonnie Schrenk Park.

 Place officer at location Hillcrest Park.

 Place officer at location South Surrey RCMP.

 The population coverage associated to the patrol officer distribution is: 76.76 %




In [25]:
waiting_plot = waiting[waiting['name'].isin( WPnames2 )]
    # plt.plot(demand['lon'], demand['lat'], 'rs')
plt.plot(waiting_plot['lon'].astype(np.float64), waiting_plot['lat'].astype(np.float64), 'bs')
mplleaflet.show()

In [38]:
#all the points covered
covered_points = []
for name in WPnames2:
    covered_points.extend(covered[name])
covered_points = list(set(covered_points))

In [39]:
#percentage of crime covered given the patrol from population
df_info = crime
df_info[df_info['DAuid'].isin(covered_points)]['incidents'].sum()/df_info['incidents'].sum()

0.7289580699176434